# Create Zipcode Map between cities and counties

In [9]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import numpy as np
import sets


/Users/along528/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: DeprecationWarning: the sets module is deprecated


## Load zip code data

In [102]:
zipdf = pd.read_csv('/Users/along528/Dropbox/insight/project/datasets/zipcodes/us_postal_codes.csv')
zipdf = zipdf.fillna(-1)
zipdf = zipdf[zipdf['Postal Code']!=-1]
zipdf

,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7
0,210,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
1,211,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
2,212,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
3,213,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
4,214,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
5,215,Portsmouth,New Hampshire,NH,Rockingham,43.0059,-71.0132,-1
6,401,Pleasantville,New York,NY,-1,41.1381,-73.7847,-1
7,501,Holtsville,New York,NY,Suffolk,40.9223,-72.6371,-1
8,544,Holtsville,New York,NY,Suffolk,40.9223,-72.6371,-1
9,1001,Agawam,Massachusetts,MA,Hampden,42.0702,-72.6227,-1


In [100]:
zipdf[zipdf['Postal Code']==6340]
zipdf[zipdf['Place Name']=='Groton']



,Postal Code,Place Name,State,State Abbreviation,County,Latitude,Longitude,Unnamed: 7
183,1450,Groton,Massachusetts,MA,Middlesex,42.6124,-71.5584,-1
195,1470,Groton,Massachusetts,MA,Middlesex,42.4464,-71.4594,-1
196,1471,Groton,Massachusetts,MA,Middlesex,42.4464,-71.4594,-1
1695,5046,Groton,Vermont,VT,Caledonia,44.2204,-72.2175,-1
2169,6340,Groton,Connecticut,CT,New London,41.3572,-72.0579,-1
2170,6349,Groton,Connecticut,CT,New London,41.3997,-72.0904,-1
4570,13073,Groton,New York,NY,Tompkins,42.5855,-76.3633,-1
25925,57445,Groton,South Dakota,SD,Brown,45.4503,-98.1058,-1


Fix specific places

In [103]:
test = pd.DataFrame(zipdf) #.ix('Postal Code',43581)
test.ix[test['Postal Code'] == 60018,'Place Name'] = 'Rosemont'
test.ix[test['Postal Code'] == 60176,'Place Name'] = 'Rosemont'
test.ix[test['Postal Code'] == 61554,'Place Name'] = 'Marquette Heights'
test.ix[test['Postal Code'] == 60015,'Place Name'] = 'Riverwoods'
test.ix[test['Postal Code'] == 60118,'Place Name'] = 'East Dundee'
test.ix[test['Postal Code'] == 28445,'Place Name'] = 'Surf City'
test.ix[test['Postal Code'] == 27244,'Place Name'] = 'Elon'
test.ix[test['Postal Code'] == 28584,'Place Name'] = 'Cape Carteret'
test.ix[test['Postal Code'] == 60527,'Place Name'] = 'Burr Ridge'
test.ix[test['Postal Code'] == 61616,'Place Name'] = 'Peoria Heights'
test.ix[test['Postal Code'] == 60073,'Place Name'] = 'Round Lake Heights'
test.ix[test['Postal Code'] == 28227,'Place Name'] = 'Mint Hill'
test.ix[test['Postal Code'] == 6066,'Place Name'] = 'Vernon'
test.ix[test['Postal Code'] == 6340,'Place Name'] = 'Groton Town'
test.ix[test['Postal Code'] == 6349,'Place Name'] = 'Groton Town'
zipdf = test

## Load Police Survey data

In [4]:
dbname = 'police_bjs'
username = 'along528'
pswd = 'password'
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)

In [5]:
surveydf = pd.read_sql('SELECT * FROM police_data_table',con)
surveydf

,SURVEYID,formtype,resptype,agcytype,agency,city,state,zipcode,swnauthemp,swnftemp,...,impgangtask,imphumntask,impterrtask,impcell,ori,csllea04_id,population,lpdsampgrp,finalwt_page1,finalwt_page2on
0,1,L,full,5,ALASKA STATE TROOOPERS,Anchorage,AK,99811,388,366,...,0,0,0,,AKAST01,10045062,,,1.088889,1.088889
1,3,L,full,5,ALABAMA DEPARTMENT OF PUBLIC,Montgomery,AL,36102,668,668,...,0,0,0,,ALAST00,10045061,,,1.088889,1.088889
2,17,L,full,5,ARKANSAS STATE POLICE,Little Rock,AR,72209,580,536,...,0,0,0,,ARASP00,10045064,,,1.088889,1.088889
3,26,L,full,5,ARIZONA DEPARTMENT OF PUBLIC,Phoenix,AZ,85005,1398,1257,...,0,0,0,,AZCCHPX,10045063,,,1.088889,1.088889
4,53,L,full,5,CALIFORNIA HIGHWAY PATROL,Sacramento,CA,94298,7507,7146,...,0,0,0,,CA03499,10045065,,,1.088889,1.088889
5,153,L,full,5,COLORADO STATE PATROL,Lakewood,CO,80215,717,717,...,0,0,0,,COCSP00,10045066,,,1.088889,1.088889
6,169,L,full,5,CONNECTICUT STATE POLICE,Middletown,CT,6457,1255,1255,...,0,0,0,,CTCSP00,10045067,,,1.088889,1.088889
7,188,L,full,5,DELAWARE STATE POLICE,Dover,DE,19903,665,677,...,0,0,0,,DEDSP00,10045068,,,1.088889,1.088889
8,213,L,full,5,FLORIDA HIGHWAY PATROL,Tallahassee,FL,32399,1669,1664,...,0,0,0,,FLFHP00,10045069,,,1.088889,1.088889
9,305,L,full,5,GEORGIA DEPT OF PUBLIC SAFETY,Atlanta,GA,30371,1232,1030,...,0,0,0,,GAGSP00,10045070,,,1.088889,1.088889


Make sure I also look at counties

After some specific fixes, there are 5 cities in CT, NC and IL from the survey data that don't match at least one zipcode

In [104]:

zipmap = {}
count = 0
for index,survey_series in surveydf.iterrows():
    survey_name = survey_series['agency']
    survey_city = survey_series['city']
    survey_state = survey_series['state']
    #if 'sherif' in survey_name.lower() or 'county' in survey_name.lower(): continue
    if survey_state!='NC' and survey_state!='IL' and survey_state!='CT': continue
    
    #if survey_state!='NC': continue # and survey_state!='IL' and survey_state!='CT': continue

    testdf = zipdf[(zipdf['Place Name']==survey_city) & (zipdf['State Abbreviation']==survey_state)]
    if survey_state not in zipmap:
        #sets_map[survey_state] = sets.Set(testdf['Postal Code'].tolist())
        zipmap[survey_state] = []
        zipmap[survey_state]+=testdf['Postal Code'].tolist()

    else:
        #sets_map[survey_state].union(sets.Set(testdf['Postal Code'].tolist()))
        zipmap[survey_state]+=testdf['Postal Code'].tolist()

    if len(testdf)==0: 
        print survey_name,survey_city,survey_state
        count+=1
    #print survey_name,survey_city,survey_state,len(testdf)
print count

SAUGET POLICE DEPT Sauget IL
WILLOWBROOK POLICE DEPT Willowbrook IL
ROUND LAKE POLICE DEPARTMENT Round Lake IL
LAUREL PARK POLICE DEPARTMENT Laurel Park NC
WHISPERING PINES POLICE DEPARTMENT Whispering Pines NC
5


In [30]:
print len(zipmap['CT'])
print len(list(sets.Set(zipmap['CT'])))


166
166


Those zipcodes in NC, CT and IL that have not been matched are

In [54]:
zip_traffic_states = zipdf[(zipdf['State Abbreviation']=='NC') |\
                           (zipdf['State Abbreviation']=='CT') |\
                           (zipdf['State Abbreviation']=='IL')]
unmatched_zip_traffic_states = zip_traffic_states[zip_traffic_states['Postal Code'].isin(list(sets.Set(zipmap['CT']))+\
                                                          list(sets.Set(zipmap['IL']))+\
                                                          list(sets.Set(zipmap['NC'])))==False]
unmatched_zip_traffic_states.to_csv('unmatched_zip.csv')